In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import combinations
pd.set_option('display.max_columns', 100)

In [2]:
data = pd.read_csv('senate_returns.csv')
data.rename(columns = {col:col.lower() for col in data.columns}, inplace = True)
data['amount_discrete'] = data['amount_discrete'].apply(lambda x: min(x, 5))
def market_cap_discrete(x):
    if x < 10 ** 8: # <= 100,000,000
        return 1
    if x < 2.5 * 10 ** 8: # <= 250,000,000
        return 2
    if x < 5 * 10 ** 8: # <= 500,000,000
        return 3
    if x < 10 ** 9: # <= 1,000,000,000
        return 4
    if x < 2.5 * 10 ** 9: # <= 2,500,000,000
        return 5
    if x < 5 * 10 ** 9: # <= 5,000,000,000
        return 6
    if x < 10 ** 10: # <= 10,000,000,000
        return 7
    if x < 2.5 * 10 ** 10: # <= 25,000,000,000
        return 8
    if x < 5 * 10 ** 10: # <= 50,000,000,000
        return 9
    if x < 10 ** 11: # <= 100,000,000,000
        return 10
    if x < 2.5 * 10 ** 11: # <= 250,000,000,000
        return 11
    if x < 5 * 10 ** 11: # <= 500,000,000,000
        return 12
    if x < 10 ** 12: # <= 1,000,000,000,000
        return 13
    return 14
data['market_cap_discrete'] = data['market_cap'].apply(market_cap_discrete)

In [3]:
committees_high = ['agriculture_nutrition_and_forestry',
 'appropriations',
 'banking_housing_and_urban_affairs',
 'commerce_science_and_transportation',
 'energy_and_natural_resources',
 'finance',
 'health_education_labor_and_pensions']
committees_medium = ['armed_services',
 'budget',
 'environment_and_public_works',
 'foreign_relations',
 'judiciary']
committees_low = [
 'homeland_security_and_governmental_affairs',
 'rules_and_administration',
 'small_business_and_entrepreneurship',
 'veterans_affairs']
len(set(committees_high + committees_medium + committees_low))

16

In [4]:
returns = [['return_{}'.format(time), 'return_prev{}'.format(time)] for time in ['1d', '5d', '1m']] \
# + [['return_abnormal_{}'.format(time), 'return_abnormal_prev{}'.format(time)] for time in ['1d', '5d', '1m']]
returns = [['return_abnormal_{}'.format(time), 'return_abnormal_prev{}'.format(time)] for time in ['1d', '5d', '1m']]
indep_vars = ['amount_discrete', 'type', 'party', 'age', 'market_cap_discrete']
committees = [committees_high] 
#[[], committees_high, committees_high + committees_medium, committees_high + committees_medium + committees_low]

# combos = []
# for i in range(len(indep_vars) + 1):
#     combos.extend(list(map(lambda t: list(t), combinations(indep_vars, i))))
models = []
for com in committees:
    for ret in returns:
        models.append(ret + indep_vars + com)
#     models.extend(list(map(lambda c: [ret] + c, combos)))
# models.extend(list(map(lambda c: c + committees, models)))
len(models)

3

In [5]:
# smf.ols('return_1d ~ Age + In_Office', data = data[['return_1d', 'amount', 'Party']].dropna()).fit().rsquared_adj

def make_model(variables):
    model_data = data[variables].dropna()
    if len(variables) == 1:
        formula = variables[0] + ' ~ 1'
    else:
        formula = variables[0] + ' ~ ' + ' + '.join(variables[1:])
    print(formula)
    model = smf.ols(formula, data = model_data).fit()
    print(model.summary())
    return formula, model.rsquared_adj

In [6]:
r2 = {}
for model in models:
    m = make_model(model)
    r2[m[0]] = m[1]

return_abnormal_1d ~ return_abnormal_prev1d + amount_discrete + type + party + age + market_cap_discrete + agriculture_nutrition_and_forestry + appropriations + banking_housing_and_urban_affairs + commerce_science_and_transportation + energy_and_natural_resources + finance + health_education_labor_and_pensions
                            OLS Regression Results                            
Dep. Variable:     return_abnormal_1d   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.984
Date:                Fri, 25 Jun 2021   Prob (F-statistic):             0.0161
Time:                        20:46:56   Log-Likelihood:                 2073.1
No. Observations:                1171   AIC:                            -4116.
Df Residuals:                    1156   BIC:                            -4040.
Df Model:                          14                   

In [7]:
max(r2.values())

0.037519736381095514